# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235543830561                   -0.50    8.0
  2   -7.250371983163       -1.83       -1.41    1.0
  3   -7.251241656789       -3.06       -2.06    4.0
  4   -7.251054575114   +   -3.73       -1.94    4.0
  5   -7.251336163377       -3.55       -2.92    3.0
  6   -7.251338459568       -5.64       -3.29    3.0
  7   -7.251338796225       -6.47       -4.28    3.0
  8   -7.251338797873       -8.78       -4.58    4.0
  9   -7.251338798674       -9.10       -5.19    2.0
 10   -7.251338798700      -10.58       -5.72    3.0
 11   -7.251338798704      -11.52       -6.07    3.0
 12   -7.251338798705      -12.02       -6.70    2.0
 13   -7.251338798705      -13.60       -7.19    4.0
 14   -7.251338798705      -14.45       -7.52    1.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.052837734241749285
[ Info: Arnoldi iteration step 2: normres = 0.32558532160094145
[ Info: Arnoldi iteration step 3: normres = 0.3739594140103874
[ Info: Arnoldi iteration step 4: normres = 0.8412931941302195
[ Info: Arnoldi iteration step 5: normres = 0.7200506542348295
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (4.30e-01, 8.78e-02, 5.20e-01, 2.35e-01, 1.51e-02)
[ Info: Arnoldi iteration step 6: normres = 0.29543813023392956
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.23e-01, 1.62e-01, 1.54e-01, 1.07e-01, 3.41e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09902590775995564
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.95e-03, 1.51e-02, 1.13e-02, 6.02e-02, 7.20e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11793180457404419
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.07